In [ ]:
%matplotlib ipympl
import os.path as op
from mayavi import mlab
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import randn
from scipy import stats as stats
import mne
from mne.datasets import fetch_fsaverage

## Set conditions to compare, method, time window and noise covariance matrix

In [ ]:
method = 'MNE'
tmin = 0.0
tmax = 0.450
conditions = ['normal','phase_rand']
noise_cov = 'merged_nc'

## Run t-test on contrast (conditions[0] - conditions[1])

In [ ]:
data_folder = '/home/zsuzsanna/Documents/MTA/source_localization/fixed_gaze/'
src_fname = '/home/zsuzsanna/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif'
## subject '181585' ecluded!
subjects = ['842608', '587631', '217720', '059694', '394107', '356349', '044846', '050120', '269257', '103183', '862169', '284297', '643438', '048298', '414822', '638838', '390744', '930517', '093925', '213103', '331536', '205515', '135230', '320268', '319897', '321319', '303786', '822816', '163753', '667207', '424174', '612370', '528213', '009833', '927179', '515155', '366394', '133288']
print('Number of participants included: ',len(subjects))

condition0_stc_data = None
condition1_stc_data = None
tstep = None

for condition in conditions:
    stc_data = None
    for subject in subjects:
        stc_file = op.join(data_folder, condition, noise_cov, method, subject)
        stc = mne.read_source_estimate(stc_file, subject)
        stc.crop(tmin, tmax)
        tstep = stc.tstep
        if np.all(stc_data) == None:
            stc_data = stc.data
        else:
            stc_data = np.dstack((stc_data, stc.data))
    print('Collected {} stc data files from {} condition'.format(stc_data.shape[2], condition))
    if condition == conditions[0]:
        condition0_stc_data = stc_data
    else:
        condition1_stc_data = stc_data
print('Subtracting {} condition from {} condition'.format(conditions[1],conditions[0]))
X = condition0_stc_data[:, :, :] - condition1_stc_data[:,:,:]
out = stats.ttest_1samp(X, 0, axis=2) ## compute across participants
ts = [out[0]]
ps = [out[1]]

## Visualization with time viewer 

In [ ]:
stc_file = op.join(data_folder, 'normal', noise_cov, method, 'normal_average')
stc = mne.read_source_estimate(stc_file, 'fsaverage').crop(tmin,tmax)
ps_for_viz = ps[0] * -1
stc.data = ps_for_viz

views = ['lat','caudal','ventral']
mlab.options.offscreen = False
tview_kwargs = dict(hemi='split', clim=dict(kind='value', lims=[-0.06, -0.04,0.0]),
                    views=views, time_unit='s', smoothing_steps=5, colorbar = True, size = 1400, time_viewer = True)
brain = stc.plot(**tview_kwargs)